<a href="https://colab.research.google.com/github/celine-setyawan/Data-Science/blob/master/scenario3/IndoNLU-IndoBERT-base-p2/pdf/split1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT = 'drive/My Drive/KULIAH PACIL/TERM 8/SKRIPSI/Resource Riset Chrysant Celine Setyawan/'
SC3_PATH = PROJECT_ROOT + 'dataset/scenario3/'
SC2_PATH = PROJECT_ROOT + 'dataset/scenario2/'
SC1_PATH = PROJECT_ROOT + 'dataset/scenario1/'

Mounted at /content/drive


In [2]:
!cp 'drive/My Drive/KULIAH PACIL/TERM 8/SKRIPSI/Resource Riset Chrysant Celine Setyawan/Code/indonlu_helper.py' .
!pip install --upgrade transformers
!pip install ipython-autotime
%load_ext autotime

import os, sys
import pandas as pd
import indonlu_helper as ih

import torch
torch.cuda.empty_cache()
import torch.nn.functional as F

from torch import optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer

     |████████████████████████████████| 2.1MB 18.1MB/s 
     |████████████████████████████████| 901kB 51.4MB/s 
     |████████████████████████████████| 3.3MB 34.1MB/s 
time: 4.29 s (started: 2021-05-06 20:19:48 +00:00)


In [3]:
# To check if GPU is running or not
!nvidia-smi
!nvcc --version

Thu May  6 20:19:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# torch.cuda.memory_summary(device=None, abbreviated=False)

time: 890 µs (started: 2021-05-06 20:19:52 +00:00)


In [5]:
!git clone https://github.com/indobenchmark/indonlu.git

Cloning into 'indonlu'...
remote: Enumerating objects: 417, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 417 (delta 7), reused 12 (delta 4), pack-reused 394
Receiving objects: 100% (417/417), 4.82 MiB | 22.55 MiB/s, done.
Resolving deltas: 100% (202/202), done.
time: 1.12 s (started: 2021-05-06 20:19:52 +00:00)


In [6]:
os.getcwd()
%ls

drive/  indonlu/  indonlu_helper.py  __pycache__/  sample_data/
time: 120 ms (started: 2021-05-06 20:19:54 +00:00)


In [7]:
%cd /content/indonlu/
%ls

/content/indonlu
CODE_OF_CONDUCT.md    examples/  predict.py    requirements.txt      utils/
CONTRIBUTING.md       LICENSE    predict.sh    run_all_tasks.sh*
dataset/              main.py    README.id.md  run_single_task.sh*
data_utils_notebook/  modules/   README.md     submission_examples/
time: 123 ms (started: 2021-05-06 20:19:54 +00:00)


In [8]:
from utils.forward_fn import forward_sequence_classification

time: 6.88 ms (started: 2021-05-06 20:19:54 +00:00)


# **Settings**

In [9]:
# Set random seed for reproducibility  
ih.set_seed(42)

time: 5.58 ms (started: 2021-05-06 20:19:54 +00:00)


In [10]:
scenario = 3
split = 1
model_dir = SC3_PATH + 'model/indonlu-indobert-base-p2/'

pretrained_model = 'indobenchmark/indobert-base-p2'
batch_size = 16
n_epochs = 25
learning_rate = 4e-5
early_stop = 12
max_norm = 10
valid_criterion = 'F1'
metrics = ih.porn_classification_metrics

# Step size and gamma will be used only for fine-tuning
# step_size = 1
# gamma = 0.9

time: 3.29 ms (started: 2021-05-06 20:19:54 +00:00)


# **Prepare Data**

In [11]:
%cd ..
%ls

/content
drive/  indonlu/  indonlu_helper.py  __pycache__/  sample_data/
time: 117 ms (started: 2021-05-06 20:19:54 +00:00)


In [12]:
train_dataset_path = SC3_PATH + f'train_split-{split}.pkl'
valid_dataset_path = SC3_PATH + f'val_split-{split}.pkl'

time: 1.07 ms (started: 2021-05-06 20:19:54 +00:00)


* `batch_size` defines the number of data we will take per batch.
* `num_workers` defines the number of workers we want to use to fetch the data in parallel.
* `shuffle` defines whether we want to take shuffle batch data or just fetch it in sequential order.

The explanation are from https://indobenchmark.github.io/tutorials/pytorch/deep%20learning/nlp/2020/10/18/basic-pytorch-en.html#background.

In [13]:
# Load Pretrained Tokenizer
tokenizer = BertTokenizer.from_pretrained(pretrained_model)

# Process the Data in the Right Format to be Fed into the Model
train_dataset = ih.PornClassificationDataset(train_dataset_path, tokenizer, scenario=scenario, lowercase=True)
valid_dataset = ih.PornClassificationDataset(valid_dataset_path, tokenizer, scenario=scenario, lowercase=True)

train_loader = ih.PornClassificationDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=batch_size, num_workers=16, shuffle=True)
valid_loader = ih.PornClassificationDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=batch_size, num_workers=16, shuffle=False)


time: 1.48 s (started: 2021-05-06 20:19:54 +00:00)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [14]:
l2i, i2l = train_dataset.label2index, train_dataset.index2label
print(l2i)
print(i2l)

{'bukan_konten_porno': 0, 'konten_porno': 1}
{0: 'bukan_konten_porno', 1: 'konten_porno'}
time: 2.3 ms (started: 2021-05-06 20:19:56 +00:00)


In [15]:
# train_dataset.tokenizer.vocab
train_dataset.data

,index,labels,text
0,828,1,jadi ayam kampus raa
1,829,1,gada dia enggak boleh ngewe sembarangan pah
2,830,1,yah tidak bisa buka bokep dongg
3,831,1,bokep asik g
4,832,1,tanpa ekspresi tanpa ekspresi kamu di plot kan...
...,...,...,...
9265,10295,0,nesv herbal perapat vagina untuk kesehatan org...
9266,10296,0,pencegahan lainnya yang dapat dilakukan adalah...
9267,10297,0,kodrat jadi perempuan yang punya vagina tidak ...
9268,10298,0,teman paramedis bercerita gender yang orang ta...


time: 20.8 ms (started: 2021-05-06 20:19:56 +00:00)


# **Load Model**

In [16]:
# Configuration
config = BertConfig.from_pretrained(pretrained_model)
config._num_labels = train_dataset.num_labels
config.id2label = i2l
config.label2id = l2i

# Instantiate model
model = BertForSequenceClassification.from_pretrained(pretrained_model, config=config)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=learning_rate )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 24 s (started: 2021-05-06 20:19:56 +00:00)


In [17]:
ih.count_param(model)

124442882

time: 5.23 ms (started: 2021-05-06 20:20:20 +00:00)


In [18]:
config

BertConfig {
  "_name_or_path": "indobenchmark/indobert-base-p2",
  "_num_labels": 2,
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "bukan_konten_porno",
    "1": "konten_porno"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "bukan_konten_porno": 0,
    "konten_porno": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 500

time: 5.99 ms (started: 2021-05-06 20:20:20 +00:00)


In [19]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

time: 8.71 ms (started: 2021-05-06 20:20:20 +00:00)


# **Fine Tuning & Evaluation**
Fine tuning here is used to compare pre-trained models. The models are fine-tuned on my specific task (porn classification task). The hyperparameters values follow the fine-tuning hyperparameter configurations written on the pre-trained models' paper).

```
@inproceedings{wilie2020indonlu,
  title={IndoNLU: Benchmark and Resources for Evaluating Indonesian Natural Language Understanding},
  author={Bryan Wilie and Karissa Vincentio and Genta Indra Winata and Samuel Cahyawijaya and X. Li and Zhi Yuan Lim and S. Soleman and R. Mahendra and Pascale Fung and Syafri Bahar and A. Purwarianti},
  booktitle={Proceedings of the 1st Conference of the Asia-Pacific Chapter of the Association for Computational Linguistics and the 10th International Joint Conference on Natural Language Processing},
  year={2020}
}
```



In [20]:
ih.train(model,
         train_loader=train_loader,
         valid_loader=valid_loader,
         optimizer=optimizer,
         forward_fn=forward_sequence_classification,
         metrics_fn=metrics,
         valid_criterion=valid_criterion,
         i2w=i2l,
         n_epochs=n_epochs,
         max_norm=max_norm,
         early_stop=early_stop,
         model_dir=model_dir,
         split=split, 
         isTuningLR=False)

  0%|          | 0/580 [00:00<?, ?it/s]



EPOCH 1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
TRAIN LOSS:0.2941 ACC:0.8700 F1:0.8663 REC:0.8700 PRE:0.8644 LR:0.00004000: 100%|██████████| 580/580 [03:29<00:00,  2.77it/s]
VALID LOSS:0.0071 ACC:1.0000 F1:1.0000 REC:1.0000 PRE:1.0000:   5%|▍         | 3/65 [00:00<00:36,  1.69it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
VALID LOSS:0.2793 A


Elapsed time so far: 0:03:40.839851 


EPOCH 2


TRAIN LOSS:0.2022 ACC:0.9230 F1:0.9224 REC:0.9230 PRE:0.9220 LR:0.00004000: 100%|██████████| 580/580 [03:23<00:00,  2.85it/s]
VALID LOSS:0.3719 ACC:0.8583 F1:0.8560 REC:0.8583 PRE:0.8543: 100%|██████████| 65/65 [00:05<00:00, 12.79it/s]
  0%|          | 0/580 [00:00<?, ?it/s]

...

Count Stop: 10

Elapsed time so far: 0:39:32.690592 


EPOCH 12


TRAIN LOSS:0.0276 ACC:0.9946 F1:0.9946 REC:0.9946 PRE:0.9946 LR:0.00004000: 100%|██████████| 580/580 [03:40<00:00,  2.63it/s]
VALID LOSS:0.8132 ACC:0.8505 F1:0.8421 REC:0.8505 PRE:0.8395: 100%|██████████| 65/65 [00:05<00:00, 12.88it/s]
  0%|          | 0/580 [00:00<?, ?it/s]

Count Stop: 11

Elapsed time so far: 0:43:18.660765 


EPOCH 13


TRAIN LOSS:0.0224 ACC:0.9956 F1:0.9956 REC:0.9956 PRE:0.9956 LR:0.00004000: 100%|██████████| 580/580 [03:42<00:00,  2.60it/s]
VALID LOSS:0.9172 ACC:0.8398 F1:0.8240 REC:0.8398 PRE:0.8233: 100%|██████████| 65/65 [00:05<00:00, 12.94it/s]

Count Stop: 12



============================== FINISH TRAINING AND VALIDATION ==============================
Total Training and Validation Time:
      0:47:06.598013
Best Performance (achieved at Epoch 0):
      ACC:0.9058 F1:0.8997 REC:0.9058 PRE:0.9030
Best Classification Report (achieved at Epoch 0):
                    precision    recall  f1-score   support

bukan_konten_porno       0.86      0.62      0.72       202
      konten_porno       0.91      0.98      0.94       828

          accuracy                           0.91      1030
         macro avg       0.89      0.80      0.83      1030
      weighted avg       0.90      0.91      0.90      1030

time: 47min 6s (started: 2021-05-06 20:20:20 +00:00)


In [21]:
# Load best model
# model.load_state_dict(torch.load(model_dir + f'best_model_split-{split}.th'))

time: 1.02 ms (started: 2021-05-06 21:07:26 +00:00)
